## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data (not in this noetbook)
- Agents: Build a chatbot that can take actions(not in this notebook)

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000189B2440400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000189B24410C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hi Krish, it's nice to meet you! \n\nThat's a fascinating role.  What kind of projects are you working on these days?  \n\nI'm always eager to learn more about how AI is being used in different industries.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 22, 'total_tokens': 77, 'completion_time': 0.1, 'prompt_time': 0.00014102, 'queue_time': 0.01435704, 'total_time': 0.10014102}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f834af12-b172-430c-8d26-302cfafd687e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 55, 'total_tokens': 77})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and that you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about your work? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 97, 'total_tokens': 134, 'completion_time': 0.067272727, 'prompt_time': 0.003467859, 'queue_time': 0.009640039, 'total_time': 0.070740586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c04d7ba6-52c6-4763-8cfa-e32e2af280c5-0', usage_metadata={'input_tokens': 97, 'output_tokens': 37, 'total_tokens': 134})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}
config

{'configurable': {'session_id': 'chat1'}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [15]:
from pprint import pprint
pprint(response.content)

("That's great to meet you, Krish! \n"
 '\n'
 'Being a Chief AI Engineer is a fascinating role. What kind of projects are '
 'you working on these days?  \n'
 '\n'
 "I'm eager to learn more about your work and the challenges you face in the "
 'field of AI.\n')


In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  😄 \n\nI remember! We just met. \n\n\n\nIs there anything else I can help you with?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 95, 'total_tokens': 125, 'completion_time': 0.054545455, 'prompt_time': 0.002743901, 'queue_time': 0.01139944, 'total_time': 0.057289356}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f54546b-e291-43c1-af3c-41075ed6e0f3-0', usage_metadata={'input_tokens': 95, 'output_tokens': 30, 'total_tokens': 125})

In [17]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
pprint(response.content)

('As an AI, I have no memory of past conversations and do not know your name. '
 "If you'd like to tell me your name, I'd be happy to know! 😊  \n"
 '\n')


In [ ]:
# We changed session id hence it will not remember the name

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John! It's nice to meet you. 😊\n\nWhat can I do for you today?  \n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John! \n\nI remember that you told me earlier. 😊  Is there anything else I can help you with? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [20]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [21]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you. \n\nWhat can I do for you today?  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 30, 'total_tokens': 57, 'completion_time': 0.049090909, 'prompt_time': 0.00014684, 'queue_time': 0.013910449, 'total_time': 0.049237749}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-662c6004-02e7-453f-ab0a-0e16d6e3e8ca-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

In [25]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x00000189B2433AC0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [27]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

pprint(response.content)

"Hi Krish!  It's nice to meet you. 😊\n\nWhat can I do for you today? \n"


In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

pprint(response.content)

('Your name is Krish! 😊  I remember.  \n'
 '\n'
 'Is there anything else I can help you with?\n')


In [29]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [34]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is bhishma")],"language":"sanskrit"})
response.content

'नमस्ते, भिष्म । \n\nभवान् भिष्म इति नातृत्वशः आह्वानं गृहीतव्यं । \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am shiva")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते शिव!  😊 \n\nबहुत अच्छा! \n'

In [40]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [41]:
response.content

'आपका नाम शिव है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [52]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=50,
    strategy="last", #focus on last conversation
    token_counter=model,
    include_system=True, #Include the system message to
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
pprint(trimmer.invoke(messages))

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]


In [53]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) #to add trimmer to the chain
    | prompt #adding prompt 
    | model #adding model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"That's a tough one! I can't actually know what flavors you like.  \n\nTo help me guess, tell me:\n\n* **Do you like fruity flavors, chocolatey flavors, or something else?**\n* **Do you prefer something rich and creamy or light and refreshing?**\n* **Are there any flavors you absolutely don't like?**\n\n\nThe more information you give me, the better I can guess your favorite ice cream! 🍦😊  \n\n"

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".  \n\nDo you want to try another one? 😊 \n'

In [55]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or personal information about you, so I don't know your name.  \n\nIf you'd like to tell me, I'd love to know! 😊\n"

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful and harmless AI, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"